# Predict the Price of an Airbnb
Before training a gradient boosting model or any machine learning model, it's important to preprocess the data to ensure it is in a suitable format and contains meaningful information.

In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sn
import numpy as np
import re
import time

In [ ]:
# Load data
df_listings=pd.read_csv('listings.csv')
print(f'df_listings shape: {df_listings.shape}')
print(df_listings.info())
df_listings.head()

## Heatmaps for Correlation Matrices for Numeric Columns

In [ ]:
from matplotlib.colors import LinearSegmentedColormap
# Create a cmap which we can use in the entire notebook

# Define the starting and ending colors as from Airbnb
start_color = '#00A699'
end_color = '#FF5A5F'

# Create a list of color stops
color_stops = [0.0, 1.0]

# Create a list of colors corresponding to the color stops
colors = [start_color, end_color]

# Create the colormap using LinearSegmentedColormap
cmap = LinearSegmentedColormap.from_list('custom_colormap', list(zip(color_stops, colors)))

In [ ]:
# Plot for all numeric review columns
corr_matrix = df_listings.filter(like='review').select_dtypes(exclude='object').corr()
sn.heatmap(corr_matrix, cmap=cmap)
plt.show()

In [ ]:
# Plot for the remaining numeric columns
corr_matrix = df_listings.filter(regex=r'^(?!.*review).*$', axis=1).select_dtypes(exclude='object').corr()
sn.heatmap(corr_matrix, cmap=cmap)

The plot shows that several features are correlated and should be dealt with. We will do this later.

In [ ]:
import math

review_cols = df_listings.filter(like='review').select_dtypes(exclude='object').columns
num_cols = len(review_cols)

num_rows = math.ceil(num_cols / 5)  # Update the number of rows and subplot grid
fig, ax = plt.subplots(num_rows, 5, figsize=(15, num_rows * 3))  # Adjust the figure size

for i, col in enumerate(review_cols):
    ax.flat[i].hist(df_listings[col], bins=50)
    title = col.replace('_', ' ').title()
    ax.flat[i].set_title(title)

# Remove empty subplots
if num_cols < num_rows * 5:  # Update the condition for removing empty subplots
    for j in range(num_cols, num_rows * 5):
        fig.delaxes(ax.flat[j])

plt.tight_layout()
plt.show()


The plots show, that most reviews range between 4 and 5 and the respective feature is skewed to the left. However, number of reviews are usually skewed to the right. Neither of the features are normally distributed. Applying log could help normalize the features.

In [ ]:
# Filter columns of specific data types
numeric_columns = df_listings.select_dtypes(include=['int', 'float']).columns
categorical_columns = df_listings.select_dtypes(include=['object']).columns
categorical_columns

The string columns contain:
* several categorical columns that need to be encoded
* several date columns: some that could be used like "host_since" and others like "calendar_last_scaped" that are less useful
* several boolean columns that need to be converted from t/f to 1/0 like "instant_bookable", e.g. using np.where
* multiple columns with percentages that need to be converted to floats like host_response_rate
* column "amenities" contains lists
# 1. Data Cleaning
## 1.1 Handling Missing Values
Check for missing values in the dataset and decide how to handle them. You can either remove rows with missing values, impute the missing values with appropriate methods (e.g., mean, median, or most frequent value), or use advanced imputation techniques if necessary.

In [ ]:
# Drop columns with all Nan-values
df_listings = df_listings.dropna(how='all', axis=1)
# Drop rows with Nan-values in target column price
df_listings = df_listings.dropna(subset='price')

In [ ]:
df_na = df_listings.isna().sum().sort_values(ascending=False).head(32).to_frame()
df_na.rename(columns = {0:'abs'}, inplace = True)
df_na['rel'] = round(df_na['abs']/len(df_listings)*100, 2)
df_na

In [ ]:
# Drop several columns of irrelevant information
df_listings.drop(axis=1, columns=['description',
                                  'host_about',
                                  'host_neighbourhood',
                                  'host_name',
                                  'host_picture_url',
                                  'host_thumbnail_url',
                                  'name',
                                  'neighborhood_overview'], inplace=True)

In [ ]:
df_na = df_listings.isna().sum().sort_values(ascending=False).head(32).to_frame()
df_na.rename(columns = {0:'abs'}, inplace = True)
df_na['rel'] = round(df_na['abs']/len(df_listings)*100, 2)
df_na

There are 32 columns with missing values:
* some don't contain valuable information and can be dropped
* some like review_scores should be averaged

Since some tree based algorithms are not sensitive to missing values lets keep one dataset with missing values.

## 1.2 Removing duplicates
Identifying and removing duplicate records to avoid bias and ensure data integrity.

In [ ]:
# Are there any duplicated rows?
df_listings.duplicated().sum()
df_listings_tree.duplicated().sum()

## 1.3 Fixing inconsistencies
Correcting inconsistencies in the data, such as formatting errors, spelling variations, or data entry mistakes.

In [ ]:
df_list = [df_listings, df_listings_tree]

In [ ]:
for i in range(len(df_list)):
    df = df_list[i]
    
    # Clean neighbourhood column
    # Convert the 'neighbourhood' column to string type
    df['neighbourhood'] = df['neighbourhood'].astype(str)

    # Apply string operations to clean and format the 'neighbourhood' column
    df['neighbourhood'] = df['neighbourhood'].str.strip().str.title()

    # Define the patterns to be replaced
    pattern_remove = r', Victoria, Australia|, Vi, Australia|, Vic, Australia|, Australia|, Victoria, Au| Vic| - Flagstaff |'
    pattern_swap_north = r'(\bNorth\b) (\w+)'
    pattern_swap_east = r'(\bEast\b) (\w+)'
    pattern_swap_south = r'(\bSouth\b) (\w+)'
    pattern_swap_west = r'(\bWest\b) (\w+)'
    pattern_swap_upper = r'(\bUpper\b) (\w+)'
    pattern_st_kilda = r'(?i)(st[.\s]*kilda|saint[.\s]*kilda)'

    # Apply regex substitution to the 'neighbourhood' column
    df['neighbourhood'] = df['neighbourhood'].str.replace(pattern_remove, '', regex=True)
    df['neighbourhood'] = df['neighbourhood'].str.replace(pattern_swap_north, r'\2 \1', regex=True)
    df['neighbourhood'] = df['neighbourhood'].str.replace(pattern_swap_east, r'\2 \1', regex=True)
    df['neighbourhood'] = df['neighbourhood'].str.replace(pattern_swap_south, r'\2 \1', regex=True)
    df['neighbourhood'] = df['neighbourhood'].str.replace(pattern_swap_west, r'\2 \1', regex=True)
    df['neighbourhood'] = df['neighbourhood'].str.replace(pattern_swap_upper, r'\2 \1', regex=True)
    df['neighbourhood'] = df['neighbourhood'].str.replace(pattern_st_kilda, 'St. Kilda', regex=True)


    df['neighbourhood'] = df['neighbourhood'].str.replace(' Melbourne ', '')
    df['neighbourhood'] = df['neighbourhood'].str.replace('，Melbourne', '')
    df['neighbourhood'] = df['neighbourhood'].str.replace(', Melbourne', '')
    df['neighbourhood'] = df['neighbourhood'].str.replace(r'\s\(Melbourne\)$|\sMelbourne\s', '', regex=True)
    df['neighbourhood'] = df['neighbourhood'].str.replace(r',\s*Melbourne\s*,', '', regex=True)
    df['neighbourhood'] = df['neighbourhood'].str.replace(r'\bMelbourne,\s*', '', regex=True)
    df['neighbourhood'] = df['neighbourhood'].str.replace(r'\s*/\s*Melbourne\b', '', regex=True)
    df['neighbourhood'] = df['neighbourhood'].str.replace(' Melbourne', '')
    df['neighbourhood'] = df['neighbourhood'].str.replace('墨尔本, 维多利亚', 'Melbourne')

    df['neighbourhood'] = df['neighbourhood'].str.replace('St EaSt. Kilda', 'St. Kilda East')
    df['neighbourhood'] = df['neighbourhood'].str.replace('St. KildaWest', 'St. Kilda West')
    df['neighbourhood'] = df['neighbourhood'].str.replace('^Kilda', 'St. Kilda', regex=True)

    df['neighbourhood'] = df['neighbourhood'].str.replace(' City', '')
    df['neighbourhood'] = df['neighbourhood'].str.replace(r'\(.*?\)', '', regex=True)
    df['neighbourhood'] = df['neighbourhood'].str.replace('St Albans', 'St. Albans')
    df['neighbourhood'] = df['neighbourhood'].str.replace('Yarra Valley, Yarra Glen, Healesville', 'Yarra Glen')
    df['neighbourhood'] = df['neighbourhood'].str.replace('Yellingbo, Yarra Valley', 'Yellingbo')
    df['neighbourhood'] = df['neighbourhood'].str.replace('Mt', 'Mount')
    df['neighbourhood'] = df['neighbourhood'].str.replace('Healesville, Mount Toolebewong', 'Healesville')
    df['neighbourhood'] = df['neighbourhood'].str.replace('Chum Creek/Healesville', 'Chum Creek')

    df['neighbourhood'] = df['neighbourhood'].str.strip()
    df.drop(axis=1, columns=['neighbourhood_cleansed'], inplace=True)

## 1.4 Addressing data format issues
Converting data types, dealing with units of measurement, or handling formatting inconsistencies.
### Extract floats from price column

In [ ]:
for i in range(len(df_list)):
    df = df_list[i]
    
    df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

### Convert columns with percentages to float

In [ ]:
for i in range(len(df_list)):
    df = df_list[i]
    
    # Select columns with object dtype
    cols_str = df.select_dtypes(include=['object']).columns

    # Iterate over the selected columns
    for col in cols_str:
        # Check if any value in the column matches the percentage pattern
        try:
            if df[col].str.match('[0-9]*\%$').any():

                # Extract the numeric part and convert to float
                df[col] = df[col].str.extract('(.*)\%')[0].astype(float)

                # Divide the values by 100 to convert to decimal and round
                df[col] = round(df[col] / 100, 2)

        # Handle AttributeError for columns containing NA values
        except AttributeError:
            pass

## 1.5 Handling outliers
Identifying and handling outliers, which are extreme values that deviate significantly from the rest of the data.

In [ ]:
from scipy.stats import zscore

threshold = 3

for i in range(len(df_list)):
    df = df_list[i]
    
    # Select non-binary and non-object columns
    columns_filtered = df.select_dtypes(exclude=['object']).columns
    binary_columns = [col for col in columns_filtered if set(df[col].unique()) == {0, 1}]
    final_columns = list(set(columns_filtered) - set(binary_columns))

    # Drop rows with zscore > threshold
    zscore_3_indices = set()
    for col in final_columns:
        col_zscores = zscore(df[col])
        zscore_3_indices.update(df[col_zscores > threshold].index)
    
    df_list[i] = df_list[i].drop(zscore_3_indices, axis=0)

In [ ]:
[print(i, df_list[i].shape) for i in range(len(df_list))]

## 2. Feature Selection
Analyze the available features and select the relevant ones for prediction. Remove any irrelevant or redundant features that do not contribute significantly to the target variable.

In [ ]:
for i in range(len(df_list)):
    df = df_list[i]
    
    # Drop URL columns
    cols_url = [col for col in df.columns if 'url' in col]
    df.drop(axis=1, columns=cols_url, inplace=True)
    # Drop scraping columns
    cols_scrape = [col for col in df.columns if 'scrape' in col]
    df.drop(axis=1, columns= cols_scrape, inplace=True)
    # Drop columns without irrelevant information
    df.drop(axis=1, inplace=True,
            columns=['host_id', 'host_location',
                     'id',
                     'longitude', 'latitude',
                     'source'])

## 3. Encoding Categorical Variables
Convert categorical variables into numerical representations so that they can be used in the model. This can be done through one-hot encoding, label encoding, or ordinal encoding, depending on the nature of the categorical variables and the requirements of the model.
### Correctly encode boolean columns

In [ ]:
cols_boolean = ['has_availability', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified',
                'instant_bookable']

for i in range(len(df_list)):
    df = df_list[i]
    
    for col in cols_boolean:
        # Encode boolean columns with t/f to 1/0 
        df[col] = np.where(df[col] == 't', 1,
                           np.where(df[col] == 'f', 0, np.nan))

        # Check if there are any NA values in the column
        if df[col].isna().any():
            # Convert the column to float if NA values are present
            df[col] = df[col].astype(float)
        else:
            # Convert the column to int8 if there are no NA values
            df[col] = df[col].astype(np.int8)

### Encode host_verifications

In [ ]:
# Define the regex to make a hard distinction between email and work_email
pattern = r"(?<!_)email"
# Define the lambda function to create the new column
lambda_email = lambda x: 1 if isinstance(x, list) and any(re.search(pattern, item) for item in x) else 0
lambda_work_email = lambda x: 1 if isinstance(x, list) and 'work_email' in x else 0
lambda_phone = lambda x: 1 if isinstance(x, list) and 'phone' in x else 0


for i in range(len(df_list)):
    df = df_list[i]
    
    # Apply the lambda functions to the desired column
    df['host_verifications_email'] = df['host_verifications'].apply(lambda_email)
    df['host_verifications_work_email'] = df['host_verifications'].apply(lambda_work_email)
    df['host_verifications_phone'] = df['host_verifications'].apply(lambda_phone)

    # Drop host_verifications column
    df.drop(axis=1, columns='host_verifications', inplace=True)
    
    df_list[i] = df

### One-hot encode remaining categorical columns

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Define the columns to be one-hot encoded
cols_categorical = ['host_response_time', 'room_type', 'property_type', 'neighbourhood']

for i in range(len(df_list)):
    # Preprocess the categorical columns
    df = df_list[i]
    for col in cols_categorical:
        df[col] = df[col].fillna('').str.lower().str.replace(' ', '_')

    # Create an instance of OneHotEncoder
    encoder = OneHotEncoder(sparse_output=False)

    # Fit and transform the selected columns
    encoded_data = encoder.fit_transform(df[cols_categorical])

    # Create a DataFrame with the encoded data
    df_encoded = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(cols_categorical))
    df_encoded.rename(columns={'host_response_time_': 'host_response_time_na'}, inplace=True)

    # Reset indices to avoid problems when concatenating
    df.reset_index(drop=True, inplace=True)
    df_encoded.reset_index(drop=True, inplace=True)

    # Concatenate the encoded DataFrame with the original DataFrame
    df = pd.concat([df, df_encoded], axis=1)

    # Drop the original categorical columns
    df.drop(columns=cols_categorical, inplace=True)

    # Assign the modified DataFrame back to the list
    df_list[i] = df

# 4. Feature Engineering
Create new features that might be more informative or relevant for the prediction task. For example, you could calculate additional derived features from existing ones or create interaction terms between variables.
### Make date columns usable by calculating durations 

In [ ]:
from datetime import datetime
# Date columns .to_datetime() and get duration in days until data retrieval
# End date from http://insideairbnb.com for Melbourne, Australia
end_date = datetime.strptime('13-03-2023', '%d-%m-%Y')

# Convert date columns to datetime and calculate the days passed from the end date
date_columns = ['host_since', 'first_review', 'last_review']

for i in range(len(df_list)):
    df = df_list[i]
    
    for col in date_columns:
        # Convert the column to datetime format
        df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')
        # Calculate the number of days passed from the end date
        df[col] = (end_date - df[col]).dt.days
        
    df_list[i] = df

### Extract floats from bathrooms_text column

In [ ]:
# Regular expression pattern to extract float numbers
pattern = r'(\d+(?:\.\d+)?)'

for i in range(len(df_list)):
    df = df_list[i]
    
    df['bathrooms'] = df['bathrooms_text'].str.extract(pattern).astype(float)

    # Map remaining text-based values
    mapping = {'Shared half-bath': 0.5, 'Half-bath': 0.5, 'Private half-bath': 0.5}
    df.loc[df['bathrooms'].isna(), 'bathrooms'] = df.loc[df['bathrooms'].isna(), 'bathrooms_text'].map(mapping)

    # Drop bathrooms_text column
    df.drop(axis=1, columns=['bathrooms_text'], inplace=True)
    
    # Assign the modified DataFrame back to df_list[i]
    df_list[i] = df

### Extract relevant amenities

In [ ]:
import pandas as pd
import re
from collections import defaultdict

# create a defaultdict to count the occurrences of amenities within df_listings['amenities']
dict_amenities = defaultdict(int)

for row in df_listings['amenities']:
    if pd.notnull(row):  # Check for NaN values
        for item in re.findall(r'"(.*?)"', row):
            dict_amenities[item.lower().strip()] += 1
        
# Sort dict_amenities by values in ascending order
dict_amenities = dict(sorted(dict_amenities.items(), key=lambda x: x[1], reverse=True))

# print first 5 key-value pairs of dict_amenities
print('dict_amenities')
count = 0
for key, value in dict_amenities.items():
    if count >= 1:
        print(f'{key}: {value}')
    count += 1
    if count == 11:
        break
print()

# List all amenities that occur at least in 0.5% of all Listings, thus capturing 99.5%
amenity_types = [key for key, value in dict_amenities.items() if value>len(df_listings)*.005]
amenity_types = sorted(amenity_types)

print('amenity_types:', amenity_types[10:20])

The types of amenities sometimes contain unnecessary information that can be summarised such as:
- 'coffee maker',
- 'coffee maker: espresso machine',
- 'coffee maker: french press',
- 'coffee maker: nespresso',
- 'coffee maker: pour-over coffee',

simply as 'coffee maker'. Different expressions for the same meaning can be accounted for using regular expressions such as r'\b(ac|air conditioning)\b' which matches AC and air conditioning. The regular expressions can also be placed in amenity_types before finally assigning the amenities using the assign_amenity() function.

### Write function to get synonyms for amenities
So that amenities with same meaning are are put together.

In [ ]:
from nltk.corpus import wordnet

def get_noun_synonyms(word):
# function to find synonyms for amenities
    synonyms = {word}
    word = word.replace(' ', '_')
    for synset in wordnet.synsets(word, pos=wordnet.NOUN):
        for lemma in synset.lemmas():
            lemma_name = lemma.name()
            if not any(c.isdigit() or c == '_' for c in lemma_name):
                synonyms.add(lemma_name)
    return synonyms

### Assign amenities using a function

In [ ]:
def assign_amenity(df):
    # function to assign amenities
    
    # Condensed amenity_types list according to frequency of dict_amenities
    # Captures and summarizes 99.5% of the occurring data
    # Partly contains regex to capture different writings or similar meanings
    
    amenity_types = [
        r'\b(ac|air conditioning)\b',
        'baby bath', 'baby safety gates', 'babysitter recommendations', 'backyard', 'baking sheet',
        'barbecue utensils', 'bathtub', 'bbq', 'beach access', 'beach essentials', 'bed linens',
        'bidet', 'bikes', 'blender', 'bluetooth sound system', 'board games', 'body soap',
        'books and reading material', 'bread maker', 'breakfast', 'building staff',
        'carbon monoxide alarm', 'ceiling fan', 'central air conditioning', 'changing table',
        r'children.*books and toys', r'children.*dinnerware',
        'cleaning available during stay', 'cleaning products', 'clothing storage', 'coffee maker',
        'conditioner', 'cooking basics', 'crib',
        'dedicated workspace', 'dining table', 'dishes and silverware', 'dishwasher',
        'drying rack for clothing',
        'elevator', 'essentials', 'ethernet connection', 'ev charger', 'exercise equipment',
        'extra pillows and blankets',
        'fire extinguisher', 'fire pit', 'fireplace', 'first aid kit',
        r'(free(.*)parking|free(.*)garage)', r'\b(?<!paid\s)(free\s)?dryer\b',
        r'\b(?<!paid\s)(free\s)?washer\b',
        'game console', 'gym',
        'hair dryer', 'hammock', 'hangers', 'heating', 'high chair', 'host greets you', 'hot tub',
        'hot water', 'hot water kettle',
        'iron',
        'keypad', 'kitchen',
        'lake access', 'laundromat nearby', 'lock on bedroom door', 'lockbox',
        'long term stays allowed', 'luggage dropoff allowed',
        'microwave', 'mini fridge', 'mosquito net',
        'outdoor dining area', 'outdoor furniture', 'outdoor shower', 'outlet covers', 'oven',
        'paid dryer', r'paid(.*)parking', 'paid washer', 'patio or balcony', 'pets allowed',
        'piano', 'ping pong table', 'pool', 'portable fans', 'portable heater',
        'private entrance', 'private living room',
        'record player', 'refrigerator', 'resort access', 'rice maker', 'room-darkening shades',
        'safe', 'sauna', 'security cameras on property', 'self check-in', 'shampoo',
        'shower gel', 'single level home', 'smart lock', 'smoke alarm', 'smoking allowed',
        'sound system', 'stove', 'sun loungers',
        'toaster', 'trash compactor', 'tv',
        'view',
        'waterfront', 'wifi', 'window guards', 'wine glasses'
    ]

    # create empty array to be filled and then concatenated to df
    amenity_results = np.zeros((len(df), len(amenity_types)), dtype=np.int8)

    # Iterate over amenity_types to create a list with column names to assign to df_amenities
    cols_amenities = []
    for i, amenity in enumerate(amenity_types):
        if amenity == r'\b(ac|air conditioning)\b':
            col_name = 'amenity_air_conditioning'
        elif amenity == r'children.*books and toys':
            col_name = 'amenity_childrens_books_and_toys'
        elif amenity == r'children.*dinnerware':
            col_name = 'amenity_childrens_dinnerware'
        elif amenity == r'\b(?<!paid\s)(free\s)?dryer\b':
            col_name = 'amenity_free_dryer'
        elif amenity == r'(free(.*)parking|free(.*)garage)':
            col_name = 'amenity_free_parking'
        elif amenity == r'\b(?<!paid\s)(free\s)?washer\b':
            col_name = 'amenity_free_washer'
        elif amenity == r'paid(.*)parking':
            col_name = 'amenity_paid_parking'
        else:
            col_name = 'amenity_' + amenity.replace(' ', '_')
        cols_amenities.append(col_name)

        # get set of synonyms for amenity
        set_syn_amenities = get_noun_synonyms(amenity)

        # iterate over df['amenities'], so that listing_amenities returns a string
        for j, listing_amenities in enumerate(df['amenities']):
            if pd.isnull(listing_amenities):
                continue  # Skip to the next iteration if it's NaN

            # Check if any of the items of set_syn_amenities occurs in listing_amenities
            if any(re.search(item, listing_amenities, re.IGNORECASE) for item in set_syn_amenities):
                # If so, assign 1 to the respective column and row
                amenity_results[j, i] = 1

    # Create DataFrame based on array amenity results with the column names cols_amenities
    df_amenities = pd.DataFrame(amenity_results, columns=cols_amenities)
    # Concat df and df_amenities
    df = pd.concat([df, df_amenities], axis=1)

    return df, cols_amenities

In [ ]:
for i in range(len(df_list)):
    df = df_list[i]
    df, cols_amenities = assign_amenity(df)

    # Drop amenities column
    df.drop(axis=1, columns='amenities', inplace=True)
    df_list[i] = df

In [ ]:
for i in range(len(df_list)):
    amenities_per_listing = round(df_list[i][cols_amenities].sum().sum()/len(df_listings_tree))
    print(f'{i}: On average {amenities_per_listing} amenities are attributed to one listing in Melbourne.')

## Inspect Correlation Between Features
The more variables used in a model, the more likely it is to overfit the training set. If the goal is to explain price, a sparse model with three features is more useful than a slightly better model with hundreds. So we keep only one of the highly correlated features. Let's look at the correlations first.

In [ ]:
# Calculate the correlation matrix
correlation_matrix = df_list[1].corr()

# Get the values that are larger than 0.8
high_correlations = correlation_matrix[correlation_matrix > 0.6].stack().reset_index()
high_correlations = high_correlations.rename(columns={'level_0': 'Feature 1', 'level_1': 'Feature 2', 0: 'Correlation'})

# Filter out the diagonal elements and NaN values
high_correlations = high_correlations[high_correlations['Feature 1'] != high_correlations['Feature 2']]
high_correlations = high_correlations.dropna()

# Drop duplicates
high_correlations.drop_duplicates(subset='Correlation', inplace=True)

# Sort by correlation in descending order
high_correlations = high_correlations.sort_values(by='Correlation', ascending=False)
high_correlations.head(10)

* The features *accommodates*, *beds* and *bedrooms* are highly correlated. Let's only keep *beds*.
* Features concerning availability are highly correlated. Let's only keep *availability_90*.
* Features concerning host_listings are highly correlated. Let's only keep *host_listings_count*.
* Features regarding the minimum and maximum nights can be used to inspect effects on housing availability. In terms of price they should play a minor role. Let's not keep them either.
* There are many features scoring the review of a listing and all are relatively correlated. Instead of keeping them all, lets create one feature that averages all the scores and drop the rest.

In [ ]:
for i in range(len(df_list)):
    df = df_list[i]
    
    df.drop(['accommodates', 'bedrooms'], axis=1, inplace=True)
    
    # Get all the availability columns and remove the one to be kept
    cols_availability = [col for col in df.columns if 'availability_' in col]
    # Drop the irrelevant columns
    cols_availability.remove('availability_90')
    df.drop(cols_availability, axis=1, inplace=True)
    
    # Get all the host_listings columns and remove the one to be kept
    cols_host_listings = [col for col in df.columns if 'listing' in col]
    cols_host_listings.remove('host_listings_count')
    # Drop the irrelevant columns
    df.drop(cols_host_listings, axis=1, inplace=True)
    
    cols_minmax_nights = [col for col in df.columns if 'night' in col]
    df.drop(cols_minmax_nights, axis=1, inplace=True)
    
    high_correlations[high_correlations['Feature 1'].str.contains('review_score', case=False)]

    cols_review_scores = [col for col in df.columns if 'review_score' in col]
    df['review_scores_mean'] = df.loc[:, cols_review_scores].mean(axis=1)
    df.drop(cols_review_scores, axis=1, inplace=True)
    
    df_list[i] = df

# 5. Data Imputation

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer

for i in range(len(df_list)):
    df = df_list[i]
    # Instantiate simple imputer
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    # Fit and transform
    df = imp_mean.fit_transform(df)
    # Recreate a dataframe from array
    df = pd.DataFrame(df, columns=df_list[i].columns)
    
    df_list[i] = df

# 5. Train-Test-Split
Split the data into training and testing sets to evaluate the model's performance on unseen data. This helps assess the model's generalization ability.

In [ ]:
for i in range(len(df_list)):
    df = df_list[i]

    # Put target column price as last
    target = df['price']
    df = df.drop('price', axis=1)
    df['price'] = target
    
    # Check if target column is last
    print(i, df.columns.get_loc('price'), len(df.columns))
    
    df_list[i] = df

df_list[0] is the dataset with deleted missing values and df_list[1] is the dataset with imputed missing values. For the train_test_split I use the larger dataset with imputed means below.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df_list[1].iloc[:, :-1],  # Features (excluding the target column)
                                                    df_list[1].iloc[:, -1],   # Target variable (last column)
                                                    test_size=0.2,
                                                    random_state=42,
                                                    shuffle=True)

# 6. Feature Scaling
Normalize or standardize numerical features to bring them to a similar scale. This helps prevent features with larger values from dominating the model during training. Common scaling methods include min-max scaling (scaling to a specific range) and standardization (scaling to have zero mean and unit variance).
### Kolmogorov-Smirnov Test for Normal Distribution

In [ ]:
from scipy.stats import kstest

normally_distributed_features = []
not_normally_distributed_features = []

# Iterate over each column in the dataframe
for column in df_listings.columns:
    # Perform Kolmogorov-Smirnov test on the column data
    _, p_value = kstest(df_listings[column].dropna(), 'norm')
    
    # Set the significance level
    alpha = 0.05
    
    # Check if the p-value is greater than the significance level
    if p_value > alpha:
        normally_distributed_features.append(column)
    else:
        not_normally_distributed_features.append(column)

print("Normally Distributed Features:")
print(normally_distributed_features)

df_listings = df_listings[not_normally_distributed_features]

### Perform Standard Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Scale the X_train and X_test
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
import pickle

# Define the data and file names
data = [X_train_scaled, X_test_scaled, y_train, y_test]
file_names = ['X_train_scaled', 'X_test_scaled', 'y_train', 'y_test']

# Save the data as pickles
for i, name in enumerate(file_names):
    with open(f'{name}.pkl', 'wb') as file:
        pickle.dump(data[i], file)

In [ ]:
import pickle

# Define the file names
file_names = ['X_train_scaled', 'X_test_scaled', 'y_train', 'y_test']

# Load the pickles
data = []
for name in file_names:
    with open(f'{name}.pkl', 'rb') as file:
        data.append(pickle.load(file))

# Assign the loaded data to variables
X_train_scaled, X_test_scaled, y_train, y_test = data

# 7. Model Selection and Training

In [ ]:
from sklearn.linear_model import BayesianRidge, ElasticNet, LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import numpy as np

list_models = [LinearRegression(), RandomForestRegressor(), SVR(), XGBRegressor(), GradientBoostingRegressor(),
               ElasticNet(), SGDRegressor(), BayesianRidge(), LGBMRegressor()]
list_model_names = ['Linear Regression', 'Random Forest Regression', 'Support Vector Regression', 'XGB Regression',
                    'Gradient Boosting Regression', 'Elastic Net', 'Stochastic Gradient Descent Regression',
                    'Bayesian Ridge', 'Light GBM Regressor']
score_dict = {model_name: {'rmse': None, 'mae': None} for model_name in list_model_names}

for i, model in enumerate(list_models):
    model_name = list_model_names[i]
    
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)

    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    
    score_dict[model_name]['rmse'] = rmse
    score_dict[model_name]['mae'] = mae
    
    print(f'{model_name} RMSE: {rmse}')
    print(f'{model_name} MAE: {mae}')

In [ ]:
import heapq
# Determine the three lowest RMSE and corresponding models
dict(heapq.nsmallest(3, rmse_dict.items(), key=lambda item: item[1]))

In [ ]:
from sklearn.metrics import make_scorer, mean_absolute_error

top3_models = [LGBMRegressor(), XGBRegressor(), RandomForestRegressor()]
top3_model_names = ['Light GBM Regressor', 'XGB Regression', 'Random Forest Regression']

mae = make_scorer(mean_absolute_error)

score_dict = {}

for i, model in enumerate(top3_models):
    train_sizes = np.linspace(0.1, 1.0, 10)

    train_sizes, train_scores, test_scores = learning_curve(estimator=model,
                                                            X=X_train_scaled,
                                                            y=y_train,
                                                            train_sizes=train_sizes,
                                                            cv=5,
                                                            #scoring='neg_mean_squared_error',
                                                            #scoring='neg_mean_absolute_error',
                                                            scoring=mae,
                                                            shuffle=True)

    # Calculate the mean and standard deviation of the scores
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    # Convert the scores to positive values and compute RMSE
#    train_scores = np.sqrt(-train_scores)
#    test_scores = np.sqrt(-test_scores)

    # Compute the mean and standard deviation of the scores
#    train_mean = np.mean(train_scores, axis=1)
#    train_std = np.std(train_scores, axis=1)
#    test_mean = np.mean(test_scores, axis=1)
#    test_std = np.std(test_scores, axis=1)
    
    score_dict[top3_model_names[i]] = {'train sizes': train_sizes,
                                       'train mean': train_mean,
                                       'test mean': test_mean,
                                       'train std': train_std,
                                       'test std': test_std}

## Plot Learning Curves
Plotting learning curves provides various information such as bias-variance trade-off, model performance, overfitting and underfitting, and model convergence. Only the top 3 models with the lowest RMSE are plotted.

In [ ]:
# Plot the learning curve
fig, ax = plt.subplots(1,3, figsize=(12,4), sharey=True)


for i, model_name in enumerate(top3_model_names):
    
    train_sizes, train_mean, test_mean, train_std, test_std = [score_dict[model_name][score] for score in score_dict[model_name]]
    
    ax[i].plot(train_sizes, train_mean, label='Training MAE')
    ax[i].plot(train_sizes, test_mean, label='Validation MAE')
    ax[i].fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
    ax[i].fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
    ax[i].set_xlabel('Training Set Size')
    ax[i].set_ylabel('MAE')
    #ax[i].set_xticks(np.arange(0,15000, 1000))
    ax[i].set_title(f'Learning Curve {top3_model_names[i]}', loc='left')
    ax[i].legend(loc='best')
    ax[i].grid(True)
    
plt.tight_layout()
plt.show()

In [ ]:
df_scores_lgbm = pd.DataFrame()

for key, item in score_dict['Light GBM Regressor'].items():
    df_scores_lgbm[key] = item

df_scores_lgbm

## Perform Recursive Feature Elimination

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler

# Train/test set generation
X_train, X_test, y_train, y_test = train_test_split(df_list[1].iloc[:, :-1],
                                                    df_list[1].iloc[:, -1],
                                                    train_size=3981,
                                                    random_state=42)

# Scale train and test sets with StandardScaler
X_train_std = StandardScaler().fit_transform(X_train)
X_test_std = StandardScaler().fit_transform(X_test)

# Init the transformer
rfe = RFE(estimator=LGBMRegressor(), n_features_to_select=20)

# Fit to the training data
_ = rfe.fit(X_train_std, y_train)

In [ ]:
X_train_rfe = df_list[1].iloc[:, :-1].copy()
X_train_rfe = X_train_rfe.loc[:, rfe.support_]

X_train_rfe.sample(5)

## Perform Recursive Feature Elemination with Cross Validation

In [ ]:
from sklearn.feature_selection import RFECV
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Train/test set generation
X_train, X_test, y_train, y_test = train_test_split(df_list[1].iloc[:, :-1],
                                                    df_list[1].iloc[:, -1],
                                                    test_size=0.2,
                                                    random_state=42)

# Scale train and test sets with StandardScaler
X_train_std = StandardScaler().fit_transform(X_train)
X_test_std = StandardScaler().fit_transform(X_test)

# Init, fit
rfecv = RFECV(
    estimator=LGBMRegressor(),
    min_features_to_select=5,
    step=5,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    cv=5,
)

_ = rfecv.fit(X_train_std, y_train)

In [ ]:
cols_rfecv = df_list[1].columns[:-1][_.support_]
df_list[1].loc[:, cols_rfecv].sample(5)

## Tune LGBM Regressor

In [ ]:
import optuna
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from optuna.integration import LightGBMPruningCallback

def objective(trial, X, y):
    param_grid = {
        'objective': 'regression',
        'metric': 'rmse', 
        'n_estimators': trial.suggest_int('n_estimators', 100, 10000, step=100),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 3000, step=20),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 200, 10000, step=100),
        'max_bin': trial.suggest_int('max_bin', 200, 300),
        'lambda_l1': trial.suggest_int('lambda_l1', 0, 100, step=5),
        'lambda_l2': trial.suggest_int('lambda_l2', 0, 100, step=5),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0, 15),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.2, 0.95, step=0.1),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.2, 0.95, step=0.1)
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_scores = np.empty(5)

    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
                   
        model = lgbm.LGBMRegressor(**param_grid)
        
        model.fit(X_train,
                  y_train,
                  eval_set=[(X_test, y_test)],
                  eval_metric='rmse',
                  callbacks=[LightGBMPruningCallback(trial, 'rmse'),
                             lgbm.early_stopping(50)])

        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        cv_scores[idx] = rmse

    return np.mean(cv_scores)

study = optuna.create_study(direction="minimize", study_name="LGBM Regressor")
func = lambda trial: objective(trial,
                               X_train_rfe, # X
                               df_list[1].iloc[:, -1], # y
                              )

study.optimize(func, n_trials=20)

In [ ]:
study.best_params

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X_train_rfe, # X
                                                    df_list[1].iloc[:, -1], # y
                                                    train_size = 3981 )
# Create the XGBoost model
model = LGBMRegressor(**study.best_params)

# Set the training set sizes for the learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

# Compute the learning curve scores
train_sizes, train_scores, test_scores = learning_curve(model,
                                                        X_train,
                                                        y_train, 
                                                        train_sizes=train_sizes, cv=5,
                                                        scoring='neg_mean_squared_error',
                                                        shuffle=True)

# Convert the scores to positive values and compute RMSE
train_scores = np.sqrt(-train_scores)
test_scores = np.sqrt(-test_scores)

# Compute the mean and standard deviation of the scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
import matplotlib.pyplot as plt

# Plot the learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Training RMSE')
plt.plot(train_sizes, test_mean, label='Validation RMSE')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
#plt.yticks(np.arange(0,300, 25))
#plt.xticks(np.arange(0,15000,1000))
plt.title('Learning Curve Tuned LGBM Regressor', loc='left')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
pd.DataFrame(zip(train_sizes, train_mean, test_mean), columns=['train sizes', 'train mean', 'test mean'])

Best practice when tuning hyperparameters are:
1. Tune n_estimators with default model
2. Tune hyperparameters optuna with n_estimators fixed at a value from step 1. This way you will be tuning hyperparameters in equal conditions.
3. Tune n_estimators once again as a final step on a model with tuned (other) hyperparameters.

In [ ]:
import optuna
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from optuna.integration import LightGBMPruningCallback

def objective(trial, X, y):
    param_grid = {
        'objective': 'regression',
        'metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 100, 10000, step=100),
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_scores = np.empty(5)

    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
                   
        model = lgbm.LGBMRegressor(**param_grid)
        
        model.fit(X_train,
                  y_train,
                  eval_set=[(X_test, y_test)],
                  eval_metric='rmse',
                  callbacks=[LightGBMPruningCallback(trial, 'rmse'),
                             lgbm.early_stopping(100)])

        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        cv_scores[idx] = rmse

    return np.mean(cv_scores)

study = optuna.create_study(direction="minimize", study_name="LGBM Regressor")
func = lambda trial: objective(trial,
                               df_list[1].iloc[:, :-1], # X
                               df_list[1].iloc[:, -1], # y
                              )

study.optimize(func, n_trials=20)

In [ ]:
study.best_params

In [ ]:
import optuna
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from optuna.integration import LightGBMPruningCallback

def objective(trial, X, y):
    param_grid = {
        'objective': 'regression',
        'metric': 'rmse',
        'n_estimators': 4700, # Set fixed now from previous study
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'num_leaves': trial.suggest_int('num_leaves', 20, 3000, step=20),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 200, 10000, step=100),
        'max_bin': trial.suggest_int('max_bin', 200, 300),
        'lambda_l1': trial.suggest_int('lambda_l1', 0, 100, step=5),
        'lambda_l2': trial.suggest_int('lambda_l2', 0, 100, step=5),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0, 15),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.2, 0.95, step=0.1),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.2, 0.95, step=0.1)
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_scores = np.empty(5)

    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
                   
        model = lgbm.LGBMRegressor(**param_grid)
        
        model.fit(X_train,
                  y_train,
                  eval_set=[(X_test, y_test)],
                  eval_metric='rmse',
                  callbacks=[LightGBMPruningCallback(trial, 'rmse'),
                             lgbm.early_stopping(100)])

        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        cv_scores[idx] = rmse

    return np.mean(cv_scores)

study = optuna.create_study(direction="minimize", study_name="LGBM Regressor")
func = lambda trial: objective(trial,
                               df_list[1].iloc[:, :-1], # X
                               df_list[1].iloc[:, -1], # y
                              )

study.optimize(func, n_trials=20)

In [ ]:
study.best_params

In [ ]:
import optuna
import lightgbm as lgbm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold
from optuna.integration import LightGBMPruningCallback

def objective(trial, X, y):
    
    param_grid = {
        'objective': 'regression',
        'metric': 'rmse',
        'n_estimators': trial.suggest_int('n_estimators', 3000, 5500, step=50),
        'learning_rate': 0.19285282614506857,
        'num_leaves': 1420,
        'max_depth': 12,
        'min_data_in_leaf': 200,
        'max_bin': 216,
        'lambda_l1': 65,
        'lambda_l2': 10,
        'min_gain_to_split': 8.42379574046222,
        'bagging_fraction': 0.6000000000000001,
        'feature_fraction': 0.8}

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_scores = np.empty(5)

    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
                   
        model = lgbm.LGBMRegressor(**param_grid)
        
        model.fit(X_train,
                  y_train,
                  eval_set=[(X_test, y_test)],
                  eval_metric='rmse',
                  callbacks=[LightGBMPruningCallback(trial, 'rmse'),
                             lgbm.early_stopping(100)])

        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        cv_scores[idx] = rmse

    return np.mean(cv_scores)

study = optuna.create_study(direction="minimize", study_name="LGBM Regressor")
func = lambda trial: objective(trial,
                               df_list[1].iloc[:, :-1], # X
                               df_list[1].iloc[:, -1], # y
                              )

study.optimize(func, n_trials=20)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

model = LGBMRegressor(**study.best_params)

# Set the training set sizes for the learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

# Compute the learning curve scores
train_sizes, train_scores, test_scores = learning_curve(model,
                                                        X_train_scaled,
                                                        y_train, 
                                                        train_sizes=train_sizes, cv=5,
                                                        scoring='neg_mean_squared_error',
                                                        shuffle=True)

# Convert the scores to positive values and compute RMSE
train_scores = np.sqrt(-train_scores)
test_scores = np.sqrt(-test_scores)

# Compute the mean and standard deviation of the scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
study.best_params

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error

params = {'n_estimators': 5400,
          'learning_rate': 0.19285282614506857,
          'num_leaves': 1420,
          'max_depth': 12,
          'min_data_in_leaf': 200,
          'max_bin': 216,
          'lambda_l1': 65,
          'lambda_l2': 10,
          'min_gain_to_split': 8.42379574046222,
          'bagging_fraction': 0.6000000000000001,
          'feature_fraction': 0.8}

# Create the XGBoost model
model = LGBMRegressor(**params)

# Set the training set sizes for the learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

# Compute the learning curve scores
train_sizes, train_scores, test_scores = learning_curve(model,
                                                        X_train_scaled,
                                                        y_train, 
                                                        train_sizes=train_sizes, cv=5,
                                                        scoring='neg_mean_squared_error',
                                                        shuffle=True)

# Convert the scores to positive values and compute RMSE
train_scores = np.sqrt(-train_scores)
test_scores = np.sqrt(-test_scores)

# Compute the mean and standard deviation of the scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
import matplotlib.pyplot as plt

# Plot the learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Training RMSE')
plt.plot(train_sizes, test_mean, label='Validation RMSE')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
plt.yticks(np.arange(0,300, 25))
plt.xticks(np.arange(0,15000,1000))
plt.title('Learning Curve Tuned LGBM Regressor', loc='left')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

rsme = np.sqrt(mean_squared_error(y_pred, y_test))

In [ ]:
rsme

## Linear Regression

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve

# Create an instance of the Linear Regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(X_train_scaled, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

# Create a learning curve plot
train_sizes, train_scores, test_scores = learning_curve(model, X_train_scaled, y_train, cv=5, scoring='neg_mean_squared_error')
train_rmse = np.sqrt(-train_scores.mean(axis=1))
test_rmse = np.sqrt(-test_scores.mean(axis=1))

plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_rmse, 'o-', color='r', label='Training RMSE')
plt.plot(train_sizes, test_rmse, 'o-', color='g', label='Validation RMSE')
plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
plt.title('Learning Curve Linear Regression', loc='left')
plt.legend(loc='best')
plt.grid(True)
plt.show()

### Random Forest Regressor

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Create the RandomForestRegressor model
model = RandomForestRegressor(n_estimators=100)

# Set the training set sizes for the learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

# Generate the learning curve by calling the learning_curve function
# It returns the training set sizes, training scores, and test scores
train_sizes, train_scores, test_scores = learning_curve(
    estimator=model,  # The random forest regressor to evaluate
    X=X_train_scaled,  # Input features (independent variables) of the training dataset
    y=y_train,  # Target variable (dependent variable) of the training dataset
    train_sizes=train_sizes,  # Array of training set sizes to use
    cv=5,  # Number of cross-validation folds or the cross-validation strategy
    scoring='neg_mean_squared_error'  # Evaluation metric used for scoring
)


# Convert the scores to positive values and compute RMSE
train_scores = np.sqrt(-train_scores)
test_scores = np.sqrt(-test_scores)

# Compute the mean and standard deviation of the scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Training RMSE')
plt.plot(train_sizes, test_mean, label='Validation RMSE')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
plt.title('Learning Curve Random Forest Regressor', loc='left')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
model

In [ ]:
model.fit(X_train_scaled, y_train)

In [ ]:
top_20_cols = pd.DataFrame(model.feature_importances_, index=df_list[1].columns[:-1]).sort_values(by=0, ascending=False).head(20).index

X_train_20, X_test_20, y_train, y_test = train_test_split(df_list[1].loc[:, top_20_cols],
                                                          df_list[1].iloc[:,-1],
                                                          test_size=0.3,
                                                          random_state=42)

## Support Vector Regressor

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error

# Assuming X contains the features and y contains the target variable

# Define the SVM model
model = SVR()

# Define the number of training samples to use for the learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

# Calculate the learning curve scores
train_sizes, train_scores, test_scores = learning_curve(model,
                                                        X_train_scaled,
                                                        y_train,
                                                        train_sizes=train_sizes,
                                                        cv=5,
                                                        scoring='neg_mean_squared_error')

# Convert the negative mean squared error scores to positive RMSE scores
train_scores = np.sqrt(-train_scores)
test_scores = np.sqrt(-test_scores)

# Calculate the mean and standard deviation of the RMSE scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training RMSE', color='blue')
plt.plot(train_sizes, test_mean, label='Validation RMSE', color='red')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='red')
plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
plt.title('Learning Curve Support Vector Regressor', loc='left')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
vmean_rmse = np.mean(train_scores, axis=1)
min_rmse = np.min(train_scores)

print(f'mean rmse: {vmean_rmse,4} \n\n'
      f'min rmse: {round(min_rmse,4)}')

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error, r2_score

# Define the XGBoost model
knn = KNeighborsRegressor(algorithm='brute')

# Define the number of training samples to use for the learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

# Calculate the learning curve scores
train_sizes, train_scores, test_scores = learning_curve(knn,
                                                        X_train_scaled,
                                                        y_train,
                                                        train_sizes=train_sizes,
                                                        cv=5,
                                                        scoring='neg_mean_squared_error')

# Convert the negative mean squared error scores to positive RMSE scores
train_scores = np.sqrt(-train_scores)
test_scores = np.sqrt(-test_scores)

# Calculate the mean and standard deviation of the RMSE scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training RMSE', color='blue')
plt.plot(train_sizes, test_mean, label='Validation RMSE', color='red')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='red')
plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
plt.title('Learning Curve KNN Regressor', loc='left')
plt.legend()
plt.grid(True)
plt.show()

## XGBoost

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error

# Define the XGBoost model
model = xgb.XGBRegressor()

# Define the number of training samples to use for the learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

# Calculate the learning curve scores
train_sizes, train_scores, test_scores = learning_curve(model,
                                                        X_train_scaled,
                                                        y_train,
                                                        train_sizes=train_sizes,
                                                        cv=5,
                                                        scoring='neg_mean_squared_error')

# Convert the negative mean squared error scores to positive RMSE scores
train_scores = np.sqrt(-train_scores)
test_scores = np.sqrt(-test_scores)

# Calculate the mean and standard deviation of the RMSE scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_mean, label='Training RMSE', color='blue')
plt.plot(train_sizes, test_mean, label='Validation RMSE', color='red')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1, color='red')
plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
plt.title('Learning Curve XGBoost Regressor', loc='left')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
vmean_rmse = np.mean(test_scores, axis=1)
min_rmse = np.min(test_scores)

print(f'mean rmse: {vmean_rmse,4} \n\n'
      f'min rmse: {round(min_rmse,4)}')

Although XGBoost produces the lowest RMSE of the tested models, the learning curve above indicates overfitting, which can be a problem for tree-based algorithms. Let's tune the model with *optuna* to avoid overfitting.

## Model Tuning
Model tuning, also known as hyperparameter optimisation, plays a critical role in refining machine learning models by systematically adjusting the model's parameters and hyperparameters to maximise its performance and improve its ability to generalise well to unseen data.

In [ ]:
import optuna
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Define the objective function for Optuna
def objective(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'num_boost_round': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 5),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0),
        
#        'reg_alpha': trial.suggest_float('reg_alpha', 0.001, 1),  # L1 regularization parameter (Lasso)
        'reg_lambda': trial.suggest_float('reg_alpha', 0.001, 1), # L2 regularization parameter (Ridge)
        
    }
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(df_list[1].iloc[:, :-1],
                                                                          df_list[1].iloc[:, -1],
                                                                          test_size=0.2,
                                                                          random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = xgb.DMatrix(train_data, label=train_target)
    dvalid = xgb.DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-rmse')
    
    # Train the model with early stopping
    model = xgb.train(param, dtrain, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = xgb.DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    rmse = mean_squared_error(valid_target, y_pred, squared=False)
    
    return rmse

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params = study.best_params
best_rmse = study.best_value
print("Best Hyperparameters: ", best_params)
print("Best RMSE: ", best_rmse)

## Rerun XGBoost With Tuned Parameters

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Create the XGBoost model
model = xgb.XGBRegressor(**best_params)

# Set the training set sizes for the learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

# Compute the learning curve scores
train_sizes, train_scores, test_scores = learning_curve(model,
                                                        X_train_scaled,
                                                        y_train, 
                                                        train_sizes=train_sizes, cv=5,
                                                        scoring='neg_mean_squared_error',
                                                        shuffle=True)

# Convert the scores to positive values and compute RMSE
train_scores = np.sqrt(-train_scores)
test_scores = np.sqrt(-test_scores)

# Compute the mean and standard deviation of the scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
# Plot the learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Training RMSE')
plt.plot(train_sizes, test_mean, label='Validation RMSE')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
plt.yticks(np.arange(0,275,25))
plt.xticks(np.arange(0,15000,1000))
plt.ylim(0,275)
plt.title('Learning Curve XGBoost Tuned')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
# Plot the learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Training RMSE')
plt.plot(train_sizes, test_mean, label='Validation RMSE')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.xlabel('Training Set Size')
plt.ylabel('Relative RMSE')
plt.yticks(np.arange(0,.475,0.05))
plt.xticks(np.arange(0,15000,1000))
plt.ylim(0,.475)
plt.title('Learning Curve XGBoost Tuned')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
import numpy as np
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train_scaled = imp_mean.fit_transform(X_train_scaled)

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LinearRegression

X_train_scaled = pd.DataFrame(X_train_scaled, columns=df_list[1].columns[:-1])

# Init, fit
rfecv = RFECV(
    estimator=XGBRegressor(**best_params),
    min_features_to_select=5,
    step=25,
    n_jobs=-1,
    scoring='neg_mean_squared_error',
    cv=5,
)

_ = rfecv.fit(X_train_scaled,
              y_train.apply(lambda x: np.log(x+1)))

In [ ]:
X_train_scaled.loc[:, rfecv.support_]

In [ ]:
X_train_scaled_rcef = _.transform(X_train_scaled)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.model_selection import learning_curve
from sklearn.metrics import mean_squared_error

# Define the XGBoost model
model = xgb.XGBRegressor()

# Define the number of training samples to use for the learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

# Calculate the learning curve scores
train_sizes, train_scores, test_scores = learning_curve(model,
                                                        X_train_scaled.loc[:, rfecv.support_],
                                                        y_train.apply(lambda x: np.log(x+1)),
                                                        train_sizes=train_sizes,
                                                        cv=5,
                                                        scoring='neg_mean_squared_error')

# Convert the negative mean squared error scores to positive RMSE scores
train_scores = np.sqrt(-train_scores)
test_scores = np.sqrt(-test_scores)

# Calculate the mean and standard deviation of the RMSE scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

In [ ]:
# Plot the learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Training RMSE')
plt.plot(train_sizes, test_mean, label='Validation RMSE')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.xlabel('Training Set Size')
plt.ylabel('Relative RMSE')
plt.yticks(np.arange(0,.5,0.05))
plt.xticks(np.arange(0,15000,1000))
plt.ylim(0,.5)
plt.title('Learning Curve XGBoost Tuned')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
X_train_scaled_rcef.shape

In [ ]:
import optuna
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Define the objective function for Optuna
def objective(trial):
    # Define the search space for hyperparameters
    param = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'num_boost_round': 100000, # Fix the boosting round and use early stopping
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 10.0),
        'min_child_weight': trial.suggest_float('min_child_weight', 0.1, 10.0),
        'lambda': trial.suggest_float('lambda', 0.1, 10.0),
        'alpha': trial.suggest_float('alpha', 0.0, 10.0)}
    
    # Split the data into further training and validation sets (three sets are preferable)
    train_data, valid_data, train_target, valid_target = train_test_split(X_train_scaled_rcef, y_train, test_size=0.2, random_state=42)
    
    # Convert the data into DMatrix format
    dtrain = xgb.DMatrix(train_data, label=train_target)
    dvalid = xgb.DMatrix(valid_data, label=valid_target)
    
    # Define the pruning callback for early stopping
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-rmse')
    
    # Train the model with early stopping
    model = xgb.train(param, dtrain, evals=[(dvalid, 'validation')], early_stopping_rounds=100, callbacks=[pruning_callback])
    
    # Make predictions on the test set
    dtest = xgb.DMatrix(valid_data)
    y_pred = model.predict(dtest)
    
    # Calculate the root mean squared error
    rmse = mean_squared_error(valid_target, y_pred, squared=False)
    
    return rmse

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100) # Control the number of trials

# Print the best hyperparameters and the best RMSE
best_params = study.best_params
best_rmse = study.best_value
print("Best Hyperparameters: ", best_params)
print("Best RMSE: ", best_rmse)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Create the XGBoost model
model = xgb.XGBRegressor(**best_params)

# Set the training set sizes for the learning curve
train_sizes = np.linspace(0.1, 1.0, 10)

# Compute the learning curve scores
train_sizes, train_scores, test_scores = learning_curve(model, X_train_scaled_rcef, y_train, train_sizes=train_sizes, cv=5,
                                                        scoring='neg_mean_squared_error', shuffle=True)

# Convert the scores to positive values and compute RMSE
train_scores = np.sqrt(-train_scores)
test_scores = np.sqrt(-test_scores)

# Compute the mean and standard deviation of the scores
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

# Plot the learning curve
plt.figure(figsize=(8, 6))
plt.plot(train_sizes, train_mean, label='Training RMSE')
plt.plot(train_sizes, test_mean, label='Validation RMSE')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)
plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, alpha=0.1)
plt.xlabel('Training Set Size')
plt.ylabel('RMSE')
plt.title('Learning Curve XGBoost Tuned')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
from sklearn.linear_model import Lasso

# Create the Lasso model
lasso_model = Lasso(alpha=1.0)

# Fit the model to the data
lasso_model.fit(X_train_scaled, y_train)

# Get the coefficients after regularization
coefficients = lasso_model.coef_

coefficients_df = pd.DataFrame(index=df_list[0].columns[:-1], data={'L1_coefficients':coefficients})
coefficients_df = coefficients_df.sort_values(by='L1_coefficients', ascending=False)
coefficients_df.head(20)

In [ ]:
col_L1 = coefficients_df.loc[coefficients_df.L1_coefficients>0].index
df_listings_L1 = df_listings.loc[:, col_L1]
df_listings_L1.loc[:, 'price'] = target

X_train_scaled, X_test_scaled, y_train, y_test = train_test_split(df_listings_L1.iloc[:, :-1],
                                                                  df_listings_L1.iloc[:,-1], 
                                                                  train_size=0.8, test_size=0.2,
                                                                  random_state=42)

There are in general two ways that you can control overfitting in XGBoost:

    The first way is to directly control model complexity.

        This includes max_depth, min_child_weight and gamma.

    The second way is to add randomness to make training robust to noise.

        This includes subsample and colsample_bytree.

        You can also reduce stepsize eta. Remember to increase num_round when you do so.


The learning curve above shows that the performance of the model is significantly improved by tuning. The training RMSE decreases up to a training set size of approximately 2,250, avoiding overfitting up to this point.

In [ ]:
from sklearn.model_selection import KFold

#Establish CV scheme
CV = KFold(n_splits=5, shuffle=True, random_state=42)

ix_training, ix_test = [], []
# Loop through each fold and append the training & test indices to the empty lists above
for fold in CV.split(df_listings):
    ix_training.append(fold[0]), ix_test.append(fold[1])

In [ ]:
X, y = df_listings.iloc[:, :-1], df_listings.iloc[:, -1]

In [ ]:
import xgboost as xgb
import shap
from sklearn.metrics import mean_squared_error

SHAP_values_per_fold = []
## Loop through each outer fold and extract SHAP values 
for i, (train_outer_ix, test_outer_ix) in enumerate(zip(ix_training, ix_test)):
    #Verbose
    print('\n------ Fold Number:',i)
    X_train, X_test = X.iloc[train_outer_ix, :], X.iloc[test_outer_ix, :]
    y_train, y_test = y.iloc[train_outer_ix], y.iloc[test_outer_ix]

    model = xgb.XGBRegressor(random_state=42) # Random state for reproducibility (same results every time)
    fit = model.fit(X_train, y_train)
    y_pred = fit.predict(X_test)
    result = mean_squared_error(y_test, y_pred)
    print('RMSE:',round(np.sqrt(result),4))

    # Use SHAP to explain predictions
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_test)
    for SHAPs in shap_values:
        SHAP_values_per_fold.append(SHAPs)

In [ ]:
SHAP_values_per_fold

In [ ]:
new_index = [ix for ix_test_fold in ix_test for ix in ix_test_fold]
shap.summary_plot(np.array(SHAP_values_per_fold), X.reindex(new_index), max_display=10)

In [ ]:
np.random.seed(1) # Reproducibility 
CV_repeats = 10
# Make a list of random integers between 0 and 10000 of length = CV_repeats to act as different data splits
random_states = np.random.randint(10000, size=CV_repeats) 

######## Use a dict to track the SHAP values of each observation per CV repitition 

shap_values_per_cv = dict()
for sample in X.index:
    ## Create keys for each sample
    shap_values_per_cv[sample] = {} 
    ## Then, keys for each CV fold within each sample
    for CV_repeat in range(CV_repeats):
        shap_values_per_cv[sample][CV_repeat] = {}

## Top 10 Features Based on Feature Importance

In [ ]:
# Use params from Optuna model tuning
params = {'eta': 0.29957346988768996, 'max_depth': 5, 'subsample': 0.7620788683085897, 'colsample_bytree': 0.5445127771106928, 'gamma': 2.7717979772086045, 'min_child_weight': 0.852634656050349, 'lambda': 5.390432625416122, 'alpha': 0.9850402641809253}

# Fit the XGB model
model = xgb.XGBRegressor(**params)
model.fit(X_train_scaled, y_train)

# Get feature importances
importances = model.feature_importances_

# Create DataFrame with importances
df_importance = pd.DataFrame(importances, index=df_listings.columns[:-1], columns=['importances'])

In [ ]:
# Get top 10 features according to their importances
df_importance.sort_values(by='importances', ascending=False).head(10)

### Plot Top 10 Features

In [ ]:
top_x = 10

# Define the official Airbnb colors
colors = ['#FF5A5F', '#00A699']

# Create a plot
fig, ax = plt.subplots(figsize=(12, 4))

# Filter and sort the DataFrame
df_plot = df_importance.sort_values('importances', ascending=False)
df_plot = df_plot[:top_x]

# Reverse the order of the DataFrame
df_plot = df_plot.iloc[::-1]
    
# Customize y_labels
y_labels = [idx.replace('_', ' ').title().replace('Of', 'of').replace('Amenity', '') for idx in df_plot.index]

# Create a horizontal bar plot
ax.barh(y_labels, df_plot['importances'] * 100, height=0.8, color=colors[0])
ax.set_xlabel('Relative importance in %')
ax.set_title(f'Top Feature Importances', loc='left')

plt.tight_layout()
plt.savefig('Top10_FeatureImportances.png')
plt.show()

### Plot Top 10 of Reviews and Amenities

In [ ]:
# Get feature importances
importances = model.feature_importances_

# Create DataFrame with importances
df_importance = pd.DataFrame(importances, index=df_listings.columns[:-1], columns=['importances'])

feature_groups = ['review', 'amenity']
top_x = 10

# Define the official Airbnb colors
colors = ['#FF5A5F', '#00A699']

# Create a plot
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

for i, feature in enumerate(feature_groups):
    # Filter and sort the DataFrame
    df_plot = df_importance.filter(axis=0, like=feature).sort_values('importances', ascending=False)
    
    # Get only top_x features
    df_plot = df_plot[:top_x]
    
    # Reverse the order of the DataFrame
    df_plot = df_plot.iloc[::-1]
    
    # Customize y_labels
    y_labels = [idx.replace('_', ' ').title().replace('Of', 'of').replace('Amenity', '') for idx in df_plot.index]

    # Create a horizontal bar plot
    ax[i].barh(y_labels, df_plot['importances'] * 100, height=0.8, color=colors[i])
    ax[i].set_xlabel('Relative importance in %')
    ax[i].set_title(f'Top {top_x} {feature.title()} Importance', loc='left')

plt.tight_layout()
plt.savefig('Top10Importance_Review_Amenity.png')
plt.show()

In [ ]:
tot_review = df_importance.filter(like='review', axis=0).sum()
tot_amenity = df_importance.filter(like='amenity', axis=0).sum()
print(f'Sum of review contribution: {round(tot_review.item()*100,2)}% \n'
      f'Sum of amenity contribution: {round(tot_amenity.item()*100,2)}%')

len(df_importance.filter(like='amenity', axis=0))

## SHAP-Plots for feature importance
SHAP (SHapley Additive exPlanations) values are a method used to explain the output of a machine learning model by assigning importance values to each input feature. These values quantify the contribution of each feature to the final prediction made by the model.

SHAP values provide a way to understand and quantify the impact of each feature on the predictions made by a machine learning model. They help to understand which features are driving the predictions and provide insight into the underlying factors influencing the model's outputs.

In [ ]:
model

In [ ]:
import shap

# Use params from optuna tuning
params = {'eta': 0.26678249922705494, 'max_depth': 9, 'subsample': 0.8827268140624525, 'colsample_bytree': 0.9770252347812884, 'gamma': 3.0813302070917885, 'min_child_weight': 9.1582640029387, 'lambda': 2.0019794387256966, 'alpha': 2.762623241915627}

# Fit the XGB model
model = xgb.XGBRegressor(**params)
model.fit(X_train_scaled, y_train)

# Initialize shap with initjs()
shap.initjs()

# Create an XGB explainer object
explainer = shap.TreeExplainer(model)

# Calculate SHAP values for all features
shap_values = explainer(X_test_scaled)

# Add feature names
shap_values.feature_names_ = df_listings.iloc[:, :-1].columns

In [ ]:
# Due to long calculation time of shap_values save a pickle for later use
import pickle

# Save shap_values as a pickle file
with open('shap_values.pkl', 'wb') as f:
    pickle.dump(shap_values, f)

In [ ]:
# Load shap_values from the pickle file
with open('shap_values.pkl', 'rb') as f:
    shap_values = pickle.load(f)

In [ ]:
# Rename features for plotting
shap_values.feature_names = [feature.replace('_',' ').replace('amenity', 'amenity:').title() for feature in shap_values.feature_names_]

# Plot SHAP beeswarm
shap.plots.beeswarm(shap_values, color=cmap, show=False)
plt.tight_layout()
plt.savefig('SHAP_Summary_General.png')

In [ ]:
import pandas as pd
# Create dataframe with SHAP-values
df_shap_values = pd.DataFrame(data=shap_values.abs.mean(0).values,
                              index=shap_values.feature_names)
# Rename column
df_shap_values.rename(columns={0:'shap_values'}, inplace=True)

df_shap_values.sort_values(by='shap_values', ascending=False).head(10)

In [ ]:
# Plot for all numeric review columns
top_10 = df_shap_values.sort_values(by='shap_values', ascending=False).head(10).index
top_10 = [item.replace(' ','_').replace('Amenity:', 'Amenity').lower() for item in top_10]

corr_matrix = df_listings[top_10].corr()
sn.heatmap(corr_matrix, cmap=cmap)
plt.show()

In [ ]:
corr_matrix

## Create Summary Plots for Top10 Amenities and Reviews

In [ ]:
cmap

In [ ]:
#shap_values.values[:,idx].shape

X_test_scaled[:, idx].shape

In [ ]:
feature_types = ['Review', 'Amenity']

for i, feature_type in enumerate(feature_types):
    idx = [shap_values.feature_names.index(item) for item in shap_values.feature_names if feature_type in item]
    
    shap.summary_plot(shap_values.values[:,idx],
                      X_test_scaled[:, idx],
                      max_display=10,
                      show=False,
                      cmap=cmap,
                      feature_names=[col.replace('_', ' ').replace('Amenity: ', '').title().strip() for col in shap_values.feature_names if feature_type in col])
    
    plt.tight_layout()
    plt.savefig(f'SHAP_Summary_{feature_type.title()}.png')
    plt.show()

## Create 3 Barplots For Random Listings

In [ ]:
import random

# Set the seed for reproducibility
random.seed(42)

# Select two random indices
random_indices = random.sample(range(shap_values.shape[0]), k=3)

# Default SHAP colors
default_pos_color = "#ff0051"
default_neg_color = "#008bfb"

# Define the official Airbnb colors
positive_color = '#FF5A5F'
negative_color = '#00A699'

fig = plt.figure()
for i, idx in enumerate(random_indices):
    ax = fig.add_subplot(int(f'13{i+1}'))
    shap.plots.bar(shap_values[idx], show=False)
    
    # Change the colormap of the artists
    for fc in plt.gcf().get_children():
        # Ignore last Rectangle
        for fcc in fc.get_children()[:-1]:
            if isinstance(fcc, matplotlib.patches.Rectangle):
                if matplotlib.colors.to_hex(fcc.get_facecolor()) == default_pos_color:
                    fcc.set_facecolor(positive_color)
                elif matplotlib.colors.to_hex(fcc.get_facecolor()) == default_neg_color:
                    fcc.set_facecolor(negative_color)
            elif isinstance(fcc, plt.Text):
                if matplotlib.colors.to_hex(fcc.get_color()) == default_pos_color:
                    fcc.set_color(positive_color)
                elif matplotlib.colors.to_hex(fcc.get_color()) == default_neg_color:
                    fcc.set_color(negative_color)

plt.gcf().set_size_inches(20, 6)
plt.tight_layout()
plt.savefig('Barplots_3_Listings.png')
plt.show()

In [ ]:
shap.plots.bar(shap_values)

In [ ]:
# we convert back to the original data
# (note we can do this because X_std is a set of univariate transformations of X)
shap_values.data = scaler.inverse_transform(shap_values.data)

In [ ]:
# Initialize your Jupyter notebook with initjs(), otherwise you will get an error message.
shap.initjs()

# Custom colors
positive_color = "#ca0020"
negative_color = "#92c5de"

shap.force_plot(shap_values[0], 
                plot_cmap = [positive_color, negative_color])

# Write in a function
def shap_plot(j):
    explainerModel = shap.TreeExplainer(model)
    shap_values_Model = explainerModel.shap_values(X_train_scaled)
    p = shap.force_plot(explainerModel.expected_value,
                        shap_values_Model[j],
                        X_train.iloc[[j]], plot_cmap=[positive_color, negative_color])
    return(p)


In [ ]:
shap_plot(155)

In [ ]:
# Put target column price as last
target = df_listings['price']
df_listings = df_listings.drop('price', axis=1)
df_listings['price'] = target

In [ ]:
shap.plots.bar(shap_values)

## Create a Waterfall Plot for a Listing

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

# Default SHAP colors
default_pos_color = "#ff0051"
default_neg_color = "#008bfb"

# Define the official Airbnb colors
positive_color = '#FF5A5F'
negative_color = '#00A699'

# Plot Waterfall Plot
shap.plots.waterfall(shap_values[0], show = False)
# Change the colormap of the artists
for fc in plt.gcf().get_children():
    for fcc in fc.get_children():
        if (isinstance(fcc, matplotlib.patches.FancyArrow)):
            if (matplotlib.colors.to_hex(fcc.get_facecolor()) == default_pos_color):
                fcc.set_facecolor(positive_color)
            elif (matplotlib.colors.to_hex(fcc.get_facecolor()) == default_neg_color):
                fcc.set_color(negative_color)
        elif (isinstance(fcc, plt.Text)):
            if (matplotlib.colors.to_hex(fcc.get_color()) == default_pos_color):
                fcc.set_color(positive_color)
            elif (matplotlib.colors.to_hex(fcc.get_color()) == default_neg_color):
                fcc.set_color(negative_color)

plt.tight_layout()                
plt.savefig('WaterfallPlotListing.png')
plt.show()

In [ ]:
import shap
import xgboost as xgb

# Use params from optuna tuning
params = {'eta': 0.26678249922705494, 'max_depth': 9, 'subsample': 0.8827268140624525, 'colsample_bytree': 0.9770252347812884, 'gamma': 3.0813302070917885, 'min_child_weight': 9.1582640029387, 'lambda': 2.0019794387256966, 'alpha': 2.762623241915627}

# Fit the XGB model
model = xgb.XGBRegressor(**params)
model.fit(X_train_scaled, y_train)

# Initialize shap with initjs()
shap.initjs()

# Create an XGB explainer object
explainer = shap.explainers.Tree(model, X_test_scaled)
#explainer = shap.TreeExplainer(model)

# Calculate SHAP values for all features
shap_values = explainer(X_test_scaled)

# Add feature names
shap_values.feature_names_ = df_listings.iloc[:, :-1].columns

shap_values.data = scaler.inverse_transform(shap_values.data)

In [ ]:
shap.force_plot(base_value=explainer.expected_value,  # The expected model output
                shap_values=shap_values[14],  # SHAP values for the instance to explain
                features=df_listings.iloc[[j],:-1],  # Features of the instance to explain
                plot_cmap=[positive_color, negative_color]  # Color map for positive and negative contributions
               )

In [ ]:
shap_values[14]